<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/610_GCOv2_ReportUpgrades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

These upgrades are *exactly* the kind of “MVPv2 → enterprise-ready” jump that makes a CEO immediately *get it*. You just added two features that most governance tools (and basically all agent demos) don’t have:

1. **A one-minute executive “one-view”** (verdict → so-what → next steps)
2. **A segment view** that lets each business owner see *their* risk + exposure

That’s not formatting. That’s **decision ergonomics**.

Here’s a tight review of what’s great, what to tweak (a few small bugs/edge cases), and how to make this bulletproof without overbuilding.

---

## What you upgraded really well

### 1) Executive “One-View” is the killer feature

This section:

* `Governance risk: HIGH/MEDIUM/LOW`
* **So what** (exposure + open cases + top agents)
* **Next steps** (1–3 actions)

…is *exactly* how execs consume information.

It turns the report from “audit document” into “operating memo.”

### 2) Segment view is a second killer feature

The table:

| Agent | Risk score | Status | Exposure |

is how you scale governance across managers. It turns your platform into:

> “Every agent has an owner and a risk profile.”

This is *portfolio governance* rather than “one agent audit.”

---

## The two biggest issues to fix (important)

### Issue A — Verdict can be wrong if `overall` is a string

You use:

```python
overall = risk_scores.get("overall_risk_score") or 0
verdict = _risk_verdict(overall)
```

If `overall_risk_score` comes through as `"96.0"` (string), Python will error when comparing to `75`.

✅ Fix: cast once, consistently:

```python
overall = float(risk_scores.get("overall_risk_score") or 0)
```

Do this in both places you assign `overall` (you do it twice).

---

### Issue B — `_build_so_what_line()` will crash on formatting if exposure is a string

You do:

```python
exposure = portfolio_rollup.get("regulatory_exposure_usd") or 0
return f"Regulatory exposure ${exposure:,.0f}..."
```

If exposure is `"955000.00"` you’ll get a formatting error.

✅ Fix:

```python
exposure = float(portfolio_rollup.get("regulatory_exposure_usd") or 0)
```

Same for `open_cases` → cast to int.

---

## Other high-value improvements (small, but makes it feel “enterprise”)

### 1) `_risk_verdict()` currently returns LOW twice

Not wrong, but it reads odd:

```python
if overall_risk_score >= 25:
    return "LOW"
return "LOW"
```

✅ Either add “MINIMAL” or keep it simple:

```python
return "MINIMAL"
```

or just keep LOW but remove the redundant threshold.

---

### 2) Segment view exposure should probably focus on **open exposure**

Right now:

```python
exposure = sum(financial_exposure_usd for all cases)
```

That means closed cases still contribute to exposure.

In most executive risk reporting, “exposure” means **current open exposure**, while “lifetime exposure” can be another metric.

✅ Better default:

* `exposure_open_usd` (open cases only)
* optionally add `exposure_total_usd`

This will align with how your triggers work and how CFOs interpret numbers.

---

### 3) Segment table should highlight “Top 5” agents rather than all agents (optional)

If you ever scale to 30 agents, the report becomes long.

You can keep all agents, but exec reports often show:

* Top 5 by risk score
* Top 5 by exposure
* Then full appendix (optional)

Not required now, but easy later.

---

### 4) “Next steps” should be **action-oriented**, not just trigger messages

Right now you reuse trigger message strings:

```python
steps.append(msg)
```

Those are informative but not directive.

Better next steps usually look like:

* “Assign owner to CASE_9005 (Marketing ethics) today”
* “Freeze marketing targeting for age_range until ethics review complete”
* “Require human approval for HRDecisionAgent until protected attribute checks pass”

You already *have* the data to do that later (case IDs, policy IDs, agents).

For MVPv2, one improvement is to map trigger types to action verbs:

* risk_threshold → “Initiate exec review”
* open_cases → “Assign case owners + SLA”
* exposure → “Engage legal/compliance on exposure”

---

## One subtle but important “trust” improvement

### The report now has two “executive summary” sections

You have:

* Executive One-View (new)
* Executive Summary (existing)

That’s not wrong, but it can feel repetitive.

✅ Rename the second section to something like:

* “Executive Detail”
* “Executive Evidence”
* “Governance Summary (Detailed)”

So it reads like:

1. **One-View (act now)**
2. **Details (why)**

---

## Quick sanity check using your real report numbers

Your report shows:

* overall risk: 96
* open cases: 2
* exposure: 955,000
* high risk agents: 5

Your new one-view line would read something like:

> Regulatory exposure $955,000; 2 open cases; SalesEnablementAgent, CustomerSupportAgent, HRDecisionAgent require immediate attention.

That’s perfect.

If you make the “top 3” agents be the *highest risk* agents (not just first 3 in the list), it gets even stronger. Right now it’s “first 3” in whatever order the rollup provides.

✅ Tiny improvement: sort by risk_score before selecting top 3.

---

## Net: what these upgrades *mean* in CEO terms

This upgrade turns your report into:

✅ **a decision memo** (not just an audit)
✅ **portfolio governance** (not single-agent compliance)
✅ **manager ownership view** (segment table)
✅ **actionable escalation** (next steps)

This is exactly the kind of UX that makes executives *use* the system.




In [ ]:
"""
Generate GCO v2 audit report (markdown) with executive sections, portfolio rollup, trends, triggers.
"""

from datetime import datetime
from typing import Any, Dict, List, Optional


def _risk_verdict(overall_risk_score: float) -> str:
    """Single headline verdict from overall risk score (0–100)."""
    if overall_risk_score >= 75:
        return "HIGH"
    if overall_risk_score >= 50:
        return "MEDIUM"
    if overall_risk_score >= 25:
        return "LOW"
    return "LOW"


def _build_so_what_line(
    portfolio_rollup: Dict[str, Any],
    governance_cases: List[Dict[str, Any]],
    risk_scores: Dict[str, Any],
) -> str:
    """One-line business impact: exposure, open cases, which agents need attention."""
    exposure = portfolio_rollup.get("regulatory_exposure_usd") or 0
    open_cases = portfolio_rollup.get("open_cases") or 0
    high_risk = portfolio_rollup.get("high_risk_agents") or []
    agents_need_attention = high_risk[:3] if high_risk else []
    agents_str = ", ".join(agents_need_attention) if agents_need_attention else "None"
    return f"Regulatory exposure ${exposure:,.0f}; {open_cases} open case(s); {agents_str} require immediate attention."


def _build_next_steps(executive_triggers: List[Dict[str, Any]], portfolio_rollup: Dict[str, Any]) -> List[str]:
    """1–3 concrete next steps for the CEO."""
    steps: List[str] = []
    for t in executive_triggers[:2]:
        msg = (t.get("message") or "").strip()
        if msg:
            steps.append(msg)
    if (portfolio_rollup.get("open_cases") or 0) > 0 and len(steps) < 3:
        steps.append("Review and assign open governance cases; target resolution within 5 business days.")
    if not steps:
        steps.append("Continue monitoring; no executive action required this run.")
    return steps[:3]


def _build_segment_rows(
    risk_scores: Dict[str, Any],
    governance_cases: List[Dict[str, Any]],
    portfolio_rollup: Dict[str, Any],
) -> List[Dict[str, Any]]:
    """Segment view: one row per agent with risk score, status, exposure."""
    agent_scores = risk_scores.get("agent_scores") or {}
    open_case_ids = set(portfolio_rollup.get("open_case_ids") or [])
    cases_by_agent: Dict[str, List[Dict[str, Any]]] = {}
    for c in governance_cases or []:
        if not isinstance(c, dict):
            continue
        agent = c.get("agent_name") or "Unknown"
        cases_by_agent.setdefault(agent, []).append(c)
    rows: List[Dict[str, Any]] = []
    for agent, score_dict in sorted(agent_scores.items()):
        risk_score = score_dict.get("risk_score") or 0
        cases = cases_by_agent.get(agent) or []
        open_for_agent = [c for c in cases if (c.get("case_id") in open_case_ids or (c.get("status") or "").lower() == "open")]
        exposure = sum((c.get("financial_exposure_usd") or 0) for c in cases)
        status = "Open case(s)" if open_for_agent else ("High risk" if risk_score >= 60 else "Monitor")
        rows.append({
            "agent": agent,
            "risk_score": risk_score,
            "status": status,
            "exposure_usd": exposure,
        })
    return rows


def generate_audit_report_v2(
    summary: Dict[str, Any],
    risk_scores: Dict[str, Any],
    risk_trends: Dict[str, Any],
    portfolio_rollup: Dict[str, Any],
    executive_triggers: List[Dict[str, Any]],
    root_cause_summary: List[Dict[str, Any]],
    prioritized_issues: List[Dict[str, Any]],
    compliance_events: List[Dict[str, Any]],
    bias_signals: List[Dict[str, Any]],
    drift_signals: List[Dict[str, Any]],
    governance_cases: List[Dict[str, Any]],
    policy_enforcement_events: List[Dict[str, Any]],
    executive_escalation_triggered: Optional[bool] = None,
) -> str:
    """
    Build full markdown audit report for CEO/executive consumption.

    Includes: one-view (headline, so-what, next steps), segment view by agent,
    executive summary, portfolio rollup, risk trends, and detailed sections.
    """
    lines: List[str] = []

    lines.append("# Governance & Compliance — Audit Report (v2)")
    lines.append("")
    lines.append(f"**Generated:** {datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')} UTC")
    lines.append("")

    # --- Executive one-view (CEO can act in under a minute) ---
    overall = risk_scores.get("overall_risk_score") or 0
    verdict = _risk_verdict(overall)
    so_what = _build_so_what_line(portfolio_rollup, governance_cases or [], risk_scores)
    next_steps = _build_next_steps(executive_triggers or [], portfolio_rollup)

    if executive_triggers or executive_escalation_triggered:
        lines.append("> **Executive attention required.** Triggers exceeded; see next steps below.")
        lines.append("")

    lines.append(f"## Governance risk: **{verdict}**")
    lines.append("")
    lines.append(f"**So what:** {so_what}")
    lines.append("")
    lines.append("**Next steps:**")
    for step in next_steps:
        lines.append(f"- {step}")
    lines.append("")
    lines.append("---")
    lines.append("")

    # Segment view by owner (business managers see their picture)
    segment_rows = _build_segment_rows(risk_scores, governance_cases or [], portfolio_rollup)
    if segment_rows:
        lines.append("## Segment view by agent")
        lines.append("")
        lines.append("| Agent | Risk score | Status | Exposure (USD) |")
        lines.append("|-------|------------|--------|----------------|")
        for r in segment_rows:
            lines.append(f"| {r.get('agent', '')} | {r.get('risk_score', 0)} | {r.get('status', '')} | {r.get('exposure_usd', 0):,.0f} |")
        lines.append("")
        lines.append("---")
        lines.append("")

    # Executive summary
    lines.append("## Executive Summary")
    lines.append("")
    overall = risk_scores.get("overall_risk_score") or 0
    trend = risk_trends.get("trend_direction", "stable")
    lines.append(f"- **Overall risk score:** {overall}")
    lines.append(f"- **Trend:** {trend}")
    snippet = portfolio_rollup.get("executive_summary_snippet")
    if snippet:
        lines.append(f"- **Summary:** {snippet}")
    lines.append("")
    if executive_triggers:
        lines.append("### Executive triggers")
        lines.append("")
        for t in executive_triggers:
            lines.append(f"- **{t.get('trigger_type', '')}:** {t.get('message', '')}")
        lines.append("")
    lines.append("---")
    lines.append("")

    # Portfolio rollup
    lines.append("## Portfolio Rollup")
    lines.append("")
    lines.append(f"- **High-risk agents:** {', '.join(portfolio_rollup.get('high_risk_agents') or []) or 'None'}")
    lines.append(f"- **Open cases:** {portfolio_rollup.get('open_cases', 0)}")
    lines.append(f"- **Regulatory exposure (USD):** {portfolio_rollup.get('regulatory_exposure_usd', 0):,.2f}")
    lines.append(f"- **Top root causes:** {', '.join(portfolio_rollup.get('top_root_causes') or [])}")
    lines.append("")
    lines.append("---")
    lines.append("")

    # Risk trends
    lines.append("## Risk Trends")
    lines.append("")
    lines.append(f"- **Latest overall risk score:** {risk_trends.get('latest_overall_risk_score', 0)}")
    if risk_trends.get("prior_overall_risk_score") is not None:
        lines.append(f"- **Prior run score:** {risk_trends.get('prior_overall_risk_score')}")
        lines.append(f"- **Delta vs prior:** {risk_trends.get('delta_vs_prior')}")
    if risk_trends.get("early_warning_flags"):
        lines.append("- **Early warning flags:** " + ", ".join(risk_trends["early_warning_flags"]))
    lines.append("")
    lines.append("---")
    lines.append("")

    # Root cause summary
    if root_cause_summary:
        lines.append("## Root Cause Summary")
        lines.append("")
        for r in root_cause_summary[:10]:
            lines.append(f"- **{r.get('root_cause', '')}** — Agent: {r.get('agent_name') or 'N/A'}, Case: {r.get('case_id') or 'N/A'}, Exposure: ${(r.get('financial_exposure_usd') or 0):,.0f}")
        lines.append("")
        lines.append("---")
        lines.append("")

    # Prioritized issues
    if prioritized_issues:
        lines.append("## Prioritized Issues")
        lines.append("")
        for i, issue in enumerate(prioritized_issues[:15], 1):
            pid = issue.get("compliance_event_id") or issue.get("event_id") or ""
            score = issue.get("priority_score") or issue.get("risk_score")
            sev = issue.get("severity", "")
            agent = issue.get("agent_name", "")
            lines.append(f"{i}. [{pid}] {agent} — severity: {sev}, priority score: {score}")
        lines.append("")
        lines.append("---")
        lines.append("")

    # Compliance events (violations)
    lines.append("## Compliance Events (Violations)")
    lines.append("")
    lines.append(f"Total: {len(compliance_events)}")
    lines.append("")
    for ev in compliance_events[:20]:
        lines.append(f"- **{ev.get('compliance_event_id')}** — {ev.get('risk_type')} | policy: {ev.get('policy_id')} | severity: {ev.get('severity')} | event: {ev.get('event_id')}")
    lines.append("")
    lines.append("---")
    lines.append("")

    # Bias signals
    if bias_signals:
        lines.append("## Bias / Fairness Signals")
        lines.append("")
        for s in bias_signals[:10]:
            lines.append(f"- **{s.get('signal_id')}** — {s.get('agent_name')} | {s.get('protected_attribute')} | delta: {s.get('delta')} | risk: {s.get('risk_level')}")
        lines.append("")
        lines.append("---")
        lines.append("")

    # Drift signals
    if drift_signals:
        lines.append("## Drift / Performance Signals")
        lines.append("")
        for s in drift_signals[:10]:
            lines.append(f"- **{s.get('signal_id')}** — {s.get('agent_name')} | metric: {s.get('metric')} | delta: {s.get('delta')} | risk: {s.get('risk_level')}")
        lines.append("")
        lines.append("---")
        lines.append("")

    # Governance cases
    if governance_cases:
        lines.append("## Governance Cases")
        lines.append("")
        for c in governance_cases[:10]:
            lines.append(f"- **{c.get('case_id')}** — {c.get('agent_name')} | {c.get('triggered_by')} | status: {c.get('status')} | risk: {c.get('risk_score')} | exposure: ${(c.get('financial_exposure_usd') or 0):,.0f}")
        lines.append("")
        lines.append("---")
        lines.append("")

    # Human overrides / enforcement
    if policy_enforcement_events:
        lines.append("## Policy Enforcement & Human Overrides")
        lines.append("")
        for e in policy_enforcement_events[:10]:
            override = "override" if e.get("override") else "no override"
            lines.append(f"- **{e.get('enforcement_id')}** — policy: {e.get('policy_id')} | {e.get('action_taken')} | {override} | resolution: {e.get('resolution_time_minutes')} min")
        lines.append("")

    return "\n".join(lines)
